In [ ]:
import pandas as pd
import os
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
from IPython.display import display
pd.options.display.max_columns = None
os. getcwd() 

In [ ]:
df = pd.read_csv('dataset.csv')

In [ ]:
#convert date format 
from datetime import datetime  
df["Class Period Start"] = pd.to_datetime(df["Class Period Start"]).dt.strftime("%Y-%m-%d")
df["Class Period End"] = pd.to_datetime(df["Class Period End"]).dt.strftime("%Y-%m-%d")
df["Filing Date"] = pd.to_datetime(df["Filing Date"]).dt.strftime("%Y-%m-%d")

#drop what is unnecesary
df=df.drop(columns=['Filing Date.1'])

In [ ]:
#industry and sector dummies
df = pd.get_dummies(df, columns=['Sector'])
df = pd.get_dummies(df, columns=['Industry'])
#monthly 
df['month'] = pd.DatetimeIndex(df['Class Period Start']).month
df = pd.get_dummies(df, columns=['month'])

In [ ]:
s_columns=[x for x in df.columns if 'Sector' in x]
len(s_columns)

In [ ]:
i_columns=[x for x in df.columns if 'Industry' in x]
len(i_columns)

In [ ]:
#distribution among sectors
#for industries, use i_columns
df_s=df[s_columns].sum().sort_values(ascending=False)
df_s=pd.DataFrame(df_s, columns=['Absolute'])
df_s['Percentage']=df_s/df.shape[0]*100
df_s

In [ ]:
#plot the sector distribution over time
dft=pd.melt(df[s_columns+['Class Period Start']],id_vars='Class Period Start')
dft=pd.pivot_table(dft, index='Class Period Start', columns='variable', values='value', aggfunc=np.sum)
dft.plot(figsize=(25,25))

In [ ]:
#resample to yearly data
dft = dft.set_index(pd.DatetimeIndex(dft.index))
df_agg=dft.resample('1Y').sum()
df_agg.plot(figsize=(25,25))

In [ ]:
#plot percentages
df_agg_perc=df_agg.div(df_agg.sum(1),axis=0)*100
df_agg_perc.plot(figsize=(25,25))

In [ ]:
#add variables 
df_BC = pd.read_excel('BC.xlsx')
df_BC.rename(columns = {'Value':'recession period'}, inplace = True) 
df_BC = df_BC.set_index(pd.DatetimeIndex(df_BC['Date']))
df_BC = df_BC.drop(columns=['Date'])
result = pd.concat([df_BC, data], axis=1, join="inner")

In [ ]:
#plot a boxplot of a column
boxplot = result.boxplot(column= ['nr of cases'],grid=True, rot=45, fontsize=15,figsize='10,20')

In [ ]:
#look at correlations 
corr = result.corr()
display(corr)

In [ ]:
#plot a heatmap of correlations 
# for covariance, use .cov()
corr = result.corr()
plt.subplots(figsize=(20,15))
ax = sn.heatmap(
    corr, 
    vmin=-1, vmax=1, center=0,
    cmap=sn.diverging_palette(20, 220, n=200),
    square=True,
    annot=True
)

ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
);

In [ ]:
#general statistics 
desc = pd.concat([
    result.describe(),
    result.agg(['skew','kurt'])
]).T
desc

In [ ]:
#run different regressions 
#linear OLS
import statsmodels.api as sm
#different specifications 
X1 = result[['volatility_index','contraction','recession_period','expansion']] 
X2 = result2[['volatility_index','housing_prices_index','recession_period','Wilshire_5000_index','peak','through','contraction','expansion','GDP','unemp']] 
Y = result['nr_of_cases']
X1 = sm.add_constant(X1)
model1 = sm.OLS(Y, X1).fit(cov_type='HAC',cov_kwds={'maxlags':6})
X2 = sm.add_constant(X2)
model2 = sm.OLS(Y, X2.astype(float)).fit(cov_type='HAC',cov_kwds={'maxlags':6})

In [ ]:
#try logit / probit
#create binary dependent variable
result['prob_of_fraud'] = np.where(result['nr_of_cases']>=1, '1', '0')
#logit
result['prob_of_fraud']=result['prob_of_fraud'].apply(pd.to_numeric)
from statsmodels.formula.api import logit
model_log = logit("prob_of_fraud ~ recession_period + volatility_index + Wilshire_5000_index + housing_prices_index + peak + through + contraction + expansion+GDP+unemp", result).fit()
from statsmodels.formula.api import probit
model_prob = probit("prob_of_fraud ~ recession_period + volatility_index + Wilshire_5000_index + housing_prices_index + peak + through + contraction + expansion + GDP+unemp", result).fit()

In [ ]:
#GLM Bayesian Linear Regression
import statsmodels.formula.api as smf
formula = 'prob_of_fraud ~ recession_period + volatility_index + Wilshire_5000_index + housing_prices_index + peak + through + contraction + expansion + GDP + unemp'
model_glm = smf.glm(formula = formula, data=result, family=sm.families.Binomial())
a = model_glm.fit()
print(a.summary())

In [ ]:
#put the results together into a Stargazer table
from stargazer.stargazer import Stargazer
stargazer = Stargazer([model1,model2,model_log,model_prob,a])
stargazer.custom_columns(['OlS w HAC errors', 'OLS w HAC and more var', 'logit','probit','GLM'],[1, 1,1,1,1,])
stargazer